In [9]:
import pandas as pd
df = pd.read_csv('stock_data.csv')
df.info()#引入并展示数据

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1760 entries, 0 to 1759
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   日期      1760 non-null   object 
 1   股票代码    1760 non-null   int64  
 2   开盘      1760 non-null   float64
 3   收盘      1760 non-null   float64
 4   最高      1760 non-null   float64
 5   最低      1760 non-null   float64
 6   成交量     1760 non-null   int64  
 7   成交额     1760 non-null   float64
 8   振幅      1760 non-null   float64
 9   涨跌幅     1760 non-null   float64
 10  涨跌额     1760 non-null   float64
 11  换手率     1760 non-null   float64
dtypes: float64(9), int64(2), object(1)
memory usage: 165.1+ KB


把日期转换成时间格式，筛选数据并进行归一化

In [10]:

columns = ['开盘', '最高', '收盘', '最低', '成交量']
df['日期']=pd.to_datetime(df['日期'])
df = df[columns]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
for col in df.columns:
    if col not in ['日期', '股票代码']:
        df[col] = scaler.fit_transform(df[col].values.reshape(-1, 1))
print(df)

            开盘        最高        收盘        最低       成交量
0     0.056881  0.055722  0.055961  0.063472  0.024771
1     0.058104  0.055123  0.052311  0.060361  0.036502
2     0.051988  0.048532  0.050487  0.056627  0.023873
3     0.051376  0.050330  0.053528  0.058494  0.036685
4     0.053823  0.050330  0.053528  0.059116  0.013934
...        ...       ...       ...       ...       ...
1755  0.183486  0.186938  0.181873  0.187306  0.391089
1756  0.181651  0.177951  0.172141  0.181705  0.334354
1757  0.171865  0.171959  0.166667  0.177349  0.242523
1758  0.168196  0.174356  0.178832  0.177971  0.254140
1759  0.179817  0.177352  0.172141  0.181083  0.202350

[1760 rows x 5 columns]


绘制这些变量随时间的变化

In [11]:
import numpy as np
def data_split(dataset, look_back=60):
    X, y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :]
        X.append(a)
        y.append(dataset[i + look_back, 3])
    return np.array(X), np.array(y)

定义神经网络并进行训练

In [ ]:
import keras
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer=Adam(), loss='mean_squared_error')